yfinance documentation: https://pypi.org/project/yfinance/

yfinance Ticker documentation: https://github.com/ranaroussi/yfinance/wiki/Ticker#history

In [149]:
import yfinance as yf
import pandas as pd
import sys

In [150]:
# Input parameters
symbol = 'VGS.AX'
period= '5y'
invest_interval = 28 # Days
recur_invest = 1000

In [151]:
# Get symbol data
hist = yf.Ticker(symbol).history(period) # Retrieves pd.DataFrame
hist[0:5] # To display some values

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2018-07-16 00:00:00+10:00,69.980003,70.000000,69.750000,69.779999,28341,0.0,0.0,0.0
2018-07-17 00:00:00+10:00,69.820000,69.959999,69.639999,69.650002,56171,0.0,0.0,0.0
2018-07-18 00:00:00+10:00,70.449997,70.540001,70.360001,70.540001,32295,0.0,0.0,0.0
2018-07-19 00:00:00+10:00,70.370003,70.370003,69.989998,70.180000,28389,0.0,0.0,0.0
2018-07-20 00:00:00+10:00,70.559998,70.709999,70.260002,70.370003,29259,0.0,0.0,0.0


In [152]:
# Get every nth average value between high and low
hist['avg'] = hist[['High', 'Low']].mean(axis=1)
invest_days = hist.iloc[0::invest_interval, :]
invest_days = invest_days['avg'].reset_index() # This makes the date columns accessible and now we are just focusing on date and avg
invest_days[0:5] # To display some values

,Date,avg
0,2018-07-16 00:00:00+10:00,69.875000
1,2018-08-23 00:00:00+10:00,71.590000
2,2018-10-02 00:00:00+10:00,73.860001
3,2018-11-09 00:00:00+11:00,69.905003
4,2018-12-19 00:00:00+11:00,64.580002


In [156]:
# Get parameters for calculations
start_date = invest_days.iloc[0, 0].date()
end_date = invest_days.iloc[-1, 0].date()
last_price = invest_days.iloc[-1, 1]

def format_dollar(dol: int) -> str:
    if dol < 0:
        dol = abs(dol)
        return '-${:,.2f}'.format(dol)
    else:
        return '${:,.2f}'.format(dol)

amount_not_invested = amount_invested = total_end_value = total_end_gain = 0
for i in range(len(invest_days)):
    date = invest_days.iloc[i, 0].date()
    avg = invest_days.iloc[i, 1]
    
    # To prevent error
    if avg <= 0:
        amount_not_invested += recur_invest
        
        avg = format_dollar(avg)
        print('Could not invest on {date} due to price being {price}.'.format(date=date, price=avg))
        continue
    
    stocks_bought = int(recur_invest // avg)
    cur_value = round(stocks_bought*avg, 2)
    
    # Some amount of money will simply not be invested because you cannot buy a portion of a stock (in this model)
    amount_not_invested += recur_invest - cur_value
    amount_invested += cur_value
    
    end_value = stocks_bought*last_price
    total_end_value += end_value
    total_end_gain += end_value - cur_value
    
    avg = format_dollar(avg)
    cur_value = format_dollar(cur_value)
    print('On {date} bought {number} of {symbol} at {price}, investing {value}'.format(
        date=date,
        number=stocks_bought,
        symbol=symbol,
        price=avg,
        value=cur_value))

amount = format_dollar(recur_invest)
last_price = format_dollar(last_price)
amount_invested = format_dollar(amount_invested)
amount_not_invested = format_dollar(amount_not_invested)
total_end_value = format_dollar(total_end_value)
total_end_gain = format_dollar(total_end_gain)
print('''
Began buying {symbol} stock on {start_date} with an interval of {interval} days per investment of {amount}.
Sold all stock on {end_date} for a price of {last_price}.
Amount of money that was not invested (cannot buy portion of a stock in this model): {not_invested}
Amount of money that was invested: {invested}
The total value of all investment: {total_value}
The total gain from investment: {total_gain}
'''.format(
    symbol=symbol,
    start_date=start_date,
    interval=invest_interval,
    amount=amount,
    end_date=end_date,
    last_price=last_price,
    not_invested=amount_not_invested,
    invested=amount_invested,
    total_value=total_end_value,
    total_gain=total_end_gain))

On 2018-07-16 bought 14 of VGS.AX at $69.88, investing $978.25
On 2018-08-23 bought 13 of VGS.AX at $71.59, investing $930.67
On 2018-10-02 bought 13 of VGS.AX at $73.86, investing $960.18
On 2018-11-09 bought 14 of VGS.AX at $69.91, investing $978.67
On 2018-12-19 bought 15 of VGS.AX at $64.58, investing $968.70
On 2019-02-01 bought 14 of VGS.AX at $67.74, investing $948.29
On 2019-03-13 bought 13 of VGS.AX at $71.58, investing $930.48
On 2019-04-24 bought 13 of VGS.AX at $74.66, investing $970.64
On 2019-06-04 bought 13 of VGS.AX at $71.60, investing $930.74
On 2019-07-15 bought 13 of VGS.AX at $76.00, investing $988.00
On 2019-08-22 bought 13 of VGS.AX at $76.30, investing $991.90
On 2019-10-01 bought 12 of VGS.AX at $78.32, investing $939.78
On 2019-11-08 bought 12 of VGS.AX at $79.53, investing $954.30
On 2019-12-18 bought 12 of VGS.AX at $82.56, investing $990.72
On 2020-01-31 bought 11 of VGS.AX at $85.69, investing $942.64
On 2020-03-11 bought 13 of VGS.AX at $75.41, investing 